In [ ]:
# I did not want to trick you but the truth is, we are NOT going to play financial markets!
# However, we will see a technique that is effectively used to model them.
# In fact, we are going to write a code for doing Monte Carlo simulations (of a simple type)
# and use it to study the self-assembly of co-polymers confined to a surface.
#
# We will use a simple "bead-spring" description of the polymer, which will be described
# as a set of beads bonded with an harmonic potential. If you want, think about a bead as if
# it was a monomer of the polymer (...although somehow this is an approximation...)
# As in normal polymers, some beads are connected by a bond, some are not. Two connected beads 
# provide a contribution to the total energy of the system given by a spring-like model, hence: 
#
# Eharmonic_(i,j) = 1/2 * kspring *  | r |**2, r = x_i - x_j
#
# where x_i and x_j are the two vectors describing the position of particle i and j and k_spring
# is the spring constant, describing in practice the stiffness of the bond between the beads.
#
# In this model, all pair of beads, whether bonded or not (and regardless of whether or not they
# belong to the same polymer) interact with an additional potential of the form ( it is called the 
# Lennard-Jones potential, and we use it here just to approximate the real interactions! ):
#
# ELJ_(i,j) = 4 * epsilon * ( ( sigma / |r| )**12 - ( sigma / |r| )**6 )
# 
# In our model, we also put a cutoff to the function, so we set ELJ = 0 if |r| > rcutoff.
# We choose rcutoff = 2.5 sigma
# 
# We are looking at co-polymers, and each copolymer is made by monomers of different type.
# For simplicity, let's assume only two types, which we will refer to as A and B.
# The parameter epsilon is related to the strength of the interaction, whereas sigma plays the
# role of the diameter of the bead.
# The value of epsilon will be different depending on the specific pair of bead types we consider.
# Thus, in our system we will have three values of epsilon: eps_AA, eps_AB = eps_BA, eps_BB,
# corresponding to the 3 different potential combinations of A and B. We will also have two different
# values of sigma, sigma_A (for an AA pair) and sigma_B (for a BB pair). For an AB pair, sigma_AB is 
# the average between sigmaA and sigmaB.
# Note that in co-polymers, self-assembly is typically driven by the fact that these three values are different
# as you will see in your simulations!
#
#
# The general ingredients for our Monte Carlo simulations are the following:
# 1) You need to know the positions of the particles in your system
# 2) You have to have a function that, given the positions of the particles, returns 
#    the energy of the system. In our case, this function is
#    Etot = sum(i,j) Eharmonic_(i,j) + ELJ(i,j), where the sum extends to all possible 
#    pairs of beads i and j in the system. Note however that Eharmonic will be zero if
#    i and j are not bonded together. Moreover, ELJ will also be zero if the distance 
#    of the beads i and j is above rcutoff.
#
# You then have to evolve the system using the following rules: 
# 1) Pick a random particle in the system
# 2) Generate a random move
# 3) Calculate the change in energy due to the random move (let's call it DE )
# 4) If DE < 0 accept the new configuration. If DE > 0 then
#    i) Calculate a = exp( - DE / kT )
#    ii) Extract a random number X between (0,1)
#    iii) If X < a: accept the new configuration, otherwise remain in the old configuration
# 5) Every "nSample" attempted steps (it does not matter if reject or accepted, it is very
#    important to do exactly this!), take a record of the system to analyse it later. 
#    By record of the system one means the particle positions, from which everything else
#    can be calculated in post processing. However, for simplicity you might want to take 
#    a record of other quantities, such as the energy. They allow you to see whether or not
#    your system has equilibrated ( ...how? What do you expect to happen? )
# 6) Go back to step 1) and reiterate for a "large number of steps, let's call it nSteps" 
#
# If you repeat the algorithm described by the previous steps, if you want to calculate a 
# thermodynamic average (that is, the average but with the correct Boltzmann weight) you only need
# to do a simple average over the configurations stored in step 5, you don't need to care about the 
# energy of each of them any more! Just to make an example, to calculate the average energy 
# you will just need to sum up the energy of all configurations and divide by their number, you do not
# need to count each proportionally to exp( -E / kT ) anymore!
#
# CODING CHALLENGE:
#
# Write a Python code to perform this simulation. If done correctly,
# the code will show you how these polymers self-assemble in different structures (and under
# which conditions...see more later).
#
# Here are a few more important instruction:
#
# 1)Your code should take as input a dictionary, defining the number and type of polymers in the system, 
#    and the number "nSteps", the number of Monte Carlo (MC) steps you will need to run the algorithm for.
#    The dictionary describing the system should have the form:
#    mySystem = { "type1" : { "number": X1, "La" : N1, "Lb" : N2 } }, where:
#    typeX is just the name you give to a certain type of polymer.
#    Each polymer of the same type is characterised by the number of beads of type A, given by La, 
#    and that of typeB, given by Lb. In total, there are a certain number of polymers of this type 
#    in your system, specified by "number"
#
# 2) The initial positions of all the beads of the system can be taken randomly (note, however, that
#    each bead should have some information regarding to what are the bead it is bonded to, in order to
#    calculate the energy). It can be mathematically
#    proven that in a MC simulation, regardless of the initial state the system will tend to the same 
#    equilibrium values!
#    However, we confine our system to a 2D plane of size Lbox * Lbox. If during the simulation a random
#    movement of the bead brings it outside of it, we immediately reject that move (this corresponds to
#    having a system where there are impenentrable walls at its boundaries)
#
# 3) A random displacement for a bead can be generated by:
#    i) choose randomly between moving in x or y direction (remember we are in 2D only)
# .  ii) choose any random number in the interval ( -dxMax, dxMax ), where dxMax is a simulation parameter
#    Your results (it can be mathematically proven!) do not depend on the choice of dxMax, although the 
#    time for the system to equilibrate will. For now, just pick dxMax = 0.2 sigma_A
#
# 4) If you have N beads in the system, there will be N*(N-1)/2 pairs to be accounted for in the calculation
#    of the energy. When N starts to be large (for example, 10**3 or 10**4) we are starting to talk about
#    millions or billions of calculations to make, which is complex even for a computer!
#    However, note that there is an energy contribution only for neighbours within rcutoff, so we don't really 
#    care about others...this provides us with a way to do things more efficiently, which is:
#    i) For each bead, we calculate which of those are its neighbours. If done "intelligently", this calculation
#    requires a number of operations that is proportional to N, not N**2. In fact, you should do it, by 
#    encoding the algorithm called "cell lists". You can find the algorithm here: 
#    https://en.wikipedia.org/wiki/Cell_lists  ( for now, you can forget about so-called "periodic boundary 
#    conditions"...but ask if you want to know more )
#    ii) Once you know the neighbours of each bead, you can calculate only the energy due to the neighbours only.
#    This step also takes a number of operations proportional to N, so that in total i) and ii) take a number 
#    of operations that also grows as N instead of N**2
#
# 5) You should be recording the value of the energy and the position of the particles every nSample steps and 
#    print them into separate files which can be later used for analysis. In fact, for the case of the position
#    you want to store not only the position of each particle but also its type. You should then program a function
#    that, given the file with the type and position of the particles, plots them as a set of points, where points
#    of different colour will correspond to beads of different type. These plots will allow you to visualize 
#    self assembly!
#    Finally, you should also have a routine that, given the file with the energy, returns a plot of the energy as a 
#    function of the MC steps performed.
#
# With the code so-defined, perform different simulations changing the values defining the system and see how (and if)
# you can observe different types of structures assemblying, and in which conditions. Besides the code, you should 
# give me representative snapshot of the system for different values of aa = sigma_A / sigma_B and 
# bb = La / Lb. I also want to see a plot, that you can report as a heatmap, of the average energy of the system 
# as a function of aa and bb.
#
# In your simulations, take all other parameters fixed to the following values:
# sigma_A = 1
# eps_AA = eps_BB = 5 kT
# eps_AB = 1 kT
# Lbox = 40
# And there should be a total of 50 polymers each containing 10 beads each (how many of type A and B depends on the
# chosen value for "bb" above).
# Note that as eps is given as a multiple of kT (the so-called thermal energy, k = Boltzmann constant and T being
# temperature), there is no need to set the temperature as the Boltzmann probability will be independent on it 
# (kT cancels out!)
# By the way, once you see the code works, if you are curious you should be experimenting with different values of eps, or 
# different concentration of polymers (total number / Lbox**2 ) and see what happens. Self-assembly will truly depend
# on all of them!
#   
# One last note: all of this can be coded in different ways but you should be trying to do this splitting the problem 
# into the necessary classes. Just to make an example, you might want to have a class "bead", which contains the 
# position of the bead, and the index of the beads bonded to it, as well as functions to calculate what is the 
# energy contribution if you move that specific bead.
# Similarly, you might want to have a class, let's call it "System" which contains as elements all the beads in your
# system, as well as data regarding the size of the box for the simulation and other routines that might allow you
# to calculate the system state and print it out...
# Thus, The first step to take is NOT to start coding but to take some time to think about how to structure the code
# and thus how to divide up the work between the members of the group (for example, each one could be coding
# a class, once its structure (data attributes and methods) has been decided collectively...but this I leave it
# up to you!
#
# GOOD LUCK and HAPPY CODING!
#